## В этом документе представлен скрипт выгрузки данных по открытому апи из metmuseum.org/

Вначале импорт необходимых библиотек и выгрузка id картин с подсолнухами.

In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
link = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/' # ссылка для выгрузки данных по объектам
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')
res = r.json()
sunflower_ids = res['objectIDs']

Создаем пустой датафрейм, куда будут загружаться данные.

In [4]:
df = pd.DataFrame(columns = ['objectID', 
                                       'title', 
                                       'artistDisplayName', 
                                       'department', 
                                       'objectBeginDate', 
                                       'objectEndDate', 
                                       'period', 
                                       'objectName', 
                                       'culture'], 
                                        )

Дальше скрипт, выгружающий данные по каждому айди из списка, который создали чуть раньше.

In [7]:
for i in sunflower_ids:
    i = str(i)
    link_true = link + i
    r_test = requests.get(link_true)
    obj_test = r_test.json()

    df_1 = pd.DataFrame(obj_test, columns = ['objectID', 
                                       'title', 
                                       'artistDisplayName', 
                                       'department', 
                                       'objectBeginDate', 
                                       'objectEndDate', 
                                       'period', 
                                       'objectName', 
                                       'culture'], 
                                        index=['objectID'])
    df = pd.concat([df, df_1])


In [8]:
df

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
objectID,436524,Sunflowers,Vincent van Gogh,European Paintings,1887,1887,,Painting,
objectID,484935,Sunflowers,Edward McKnight Kauffer,Modern and Contemporary Art,1921,1921,,Painting,
objectID,437112,Bouquet of Sunflowers,Claude Monet,European Paintings,1881,1881,,Painting,
objectID,210191,Vase with sunflowers,Rozenburg Plateelfabriek,European Sculpture and Decorative Arts,1896,1896,,Vase,
objectID,431264,Sunflowers,Lily Converse,Drawings and Prints,1941,1941,,Print,
...,...,...,...,...,...,...,...,...,...
objectID,437980,Cypresses,Vincent van Gogh,European Paintings,1889,1889,,Painting,
objectID,724844,Design for Embroidery with Two Large Garlands ...,Italian School,Drawings and Prints,1845,1905,,Drawing Ornament & Architecture,
objectID,834585,"A Life in a Year–The Garden–Noon, from ""Pictur...",Arthur Boyd Houghton,Drawings and Prints,1866,1874,,Print,
objectID,834580,"A Life in a Year–The Garden–Noon, from ""Pictur...",Arthur Boyd Houghton,Drawings and Prints,1866,1874,,Print,


Проанализируем, к какой культуре относятся выгруженные объекты

In [10]:
df.culture.value_counts()

                    71
American             9
Japan                1
British              1
Ancestral Pueblo     1
Name: culture, dtype: int64

Далее чуть более сложный запрос. Выгрузим данные по объектам музея из департамента "Asian art" (departmentId = 6) с тегом "cats". Возьмем данные о первой тысяче объектов.

In [18]:
df_for_cats = pd.DataFrame(columns = ['objectID', 
                                       'title', 
                                       'artistDisplayName', 
                                       'department', 
                                       'objectBeginDate', 
                                       'objectEndDate', 
                                       'period', 
                                       'objectName', 
                                       'culture'], 
                                        )

In [17]:
r_ = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = r.json()
cats_list = cats['objectIDs']

In [13]:
for i in cats_list[0:1000]:
    i = str(i)
    link_true = link + i
    r_test = requests.get(link_true)
    obj_test = r_test.json()

    df_1 = pd.DataFrame(obj_test, columns = ['objectID', 
                                       'title', 
                                       'artistDisplayName', 
                                       'department', 
                                       'objectBeginDate', 
                                       'objectEndDate', 
                                       'period', 
                                       'objectName', 
                                       'culture'], 
                                        index=['objectID'])
    df_for_cats = pd.concat([df_for_cats, df_1])

Посмотрим, к какому периоду относятся объекты с тегом "cats".

In [19]:
df_for_cats.period.value_counts()

Edo period (1615–1868)                                   448
                                                         181
Qing dynasty (1644–1911)                                  56
Ming dynasty (1368–1644)                                  39
Muromachi period (1392–1573)                              30
                                                        ... 
Mughal period (1526–1858)                                  1
Muromachi (1392–1573)–Edo period (1615–1868)               1
Asuka period (538–710)                                     1
late Northern Song (960–1127)–Jin (1115–1234) dynasty      1
late Edo (1615–1868)-early Meiji period (1868–1912)        1
Name: period, Length: 109, dtype: int64